In [1]:
! pip3 install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [73]:
import glob
import os
import boto3
import socket
from botocore.handlers import disable_signing
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings("ignore")

In [4]:
if socket.gethostname() == 'Rohits-MBP':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/data_parquet/'
else: 
    rootdir = '' # Enter your hone dir here

In [5]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
files_with_size = [ file_path for file_path in list_of_files ]

In [6]:
dfs_parquet = {}
for file_path in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')

Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet


In [25]:
dfs_parquet['Thuesday-20-02-2018'].drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1, inplace=True)

In [7]:
for k,df in dfs_parquet.items():
    print(k)
    df['is_allowed'] = df['Label'] == 'Benign'
    del df['Label']
    print(f'DF:{k}, Shape{df.shape}')

Wednesday-21-02-2018
DF:Wednesday-21-02-2018, Shape(1048575, 80)
Friday-23-02-2018
DF:Friday-23-02-2018, Shape(1048575, 80)
Thuesday-20-02-2018
DF:Thuesday-20-02-2018, Shape(7948748, 84)
Thursday-22-02-2018
DF:Thursday-22-02-2018, Shape(1048575, 80)
Friday-16-02-2018
DF:Friday-16-02-2018, Shape(1048574, 80)
Wednesday-28-02-2018
DF:Wednesday-28-02-2018, Shape(613071, 80)
Wednesday-14-02-2018
DF:Wednesday-14-02-2018, Shape(1048575, 80)
Thursday-15-02-2018
DF:Thursday-15-02-2018, Shape(1048575, 80)
Thursday-01-03-2018
DF:Thursday-01-03-2018, Shape(331100, 80)
Friday-02-03-2018
DF:Friday-02-03-2018, Shape(1048575, 80)


In [9]:
cat_cols = ['Dst Port']

In [124]:
df_train = pd.concat([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Friday-16-02-2018'], 
                      dfs_parquet['Friday-23-02-2018'],
                      dfs_parquet['Thursday-01-03-2018'],
                      dfs_parquet['Thursday-15-02-2018'],
                      dfs_parquet['Thursday-22-02-2018'],
#                       dfs_parquet['Thuesday-20-02-2018'],
                      dfs_parquet['Wednesday-14-02-2018'],
                      dfs_parquet['Wednesday-21-02-2018']
                     ]
                    )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train['Date'] = pd.to_datetime(df_train['Timestamp']).dt.date
df_train['TS_relative'] = (df_train['Timestamp'].astype(int) - 
                             pd.to_datetime(df_train['Date']).astype(int))/ 10**9
df_train = df_train.drop(['Timestamp'], axis = 1)
df_train = df_train.drop(['Date'], axis = 1)
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train.fillna(0, inplace=True)
df_train[cat_cols] = df_train[cat_cols].astype('category')
X_train = df_train.drop(['is_allowed'], axis = 1)
y_train = df_train['is_allowed']*1 

In [123]:
def transformations(df_list):
    df_temp = pd.concat(df_list)
    df_temp['Timestamp'] = pd.to_datetime(df_temp['Timestamp'])
    df_temp['Date'] = pd.to_datetime(df_temp['Timestamp']).dt.date
    df_temp['TS_relative'] = (df_temp['Timestamp'].astype(int) - 
                             pd.to_datetime(df_temp['Date']).astype(int))/ 10**9
    df_temp = df_temp.drop(['Timestamp'], axis = 1)
    df_temp = df_temp.drop(['Date'], axis = 1)
    df_temp.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_temp.fillna(0, inplace=True)
    df_temp[cat_cols] = df_temp[cat_cols].astype('category')
    X_temp = df_temp.drop(['is_allowed'], axis = 1)
    y_temp = df_temp['is_allowed']*1 
    
    return X_temp, y_temp

In [138]:
def scale_data(df):
    stdsc = MinMaxScaler()
    stdsc.fit(df)
    return stdsc.transform(df)

In [136]:
X_train, y_train = transformations([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Friday-16-02-2018'], 
                      dfs_parquet['Friday-23-02-2018'],
                      dfs_parquet['Thursday-01-03-2018'],
                      dfs_parquet['Thursday-15-02-2018'],
                      dfs_parquet['Thursday-22-02-2018'],
                      dfs_parquet['Thuesday-20-02-2018'],
                      dfs_parquet['Wednesday-14-02-2018'],
                      dfs_parquet['Wednesday-21-02-2018']
                     ])

In [139]:
X_train_scaled = scale_data(X_train)

In [140]:
X_train.shape

(15619872, 79)

In [125]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [ ]:
iterations = 10

In [126]:
# df_test = pd.concat([dfs_parquet['Wednesday-14-02-2018'],
#                     dfs_parquet['Wednesday-21-02-2018'],
#                     dfs_parquet['Wednesday-28-02-2018']]
#                      )
df_test = dfs_parquet['Wednesday-28-02-2018']
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Date'] = pd.to_datetime(df_test['Timestamp']).dt.date
df_test['TS_relative'] = (df_test['Timestamp'].astype(int) - 
                             pd.to_datetime(df_test['Date']).astype(int))/ 10**9
df_test = df_test.drop(['Timestamp'], axis = 1)
df_test = df_test.drop(['Date'], axis = 1)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
df_test[cat_cols] = df_test[cat_cols].astype('category')
X_test = df_test.drop(['is_allowed'], axis = 1)
y_test = df_test['is_allowed'].astype(int)*1

In [127]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)

### Model Training Bayes

In [141]:
model = nn.Sequential(
    nn.Linear(in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [142]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([15619872, 79])

In [143]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([15619872])

In [131]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([613071, 79])

In [132]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([613071])

In [135]:
kl_weight = 0.1
for step in range(100):
    pre = model(x_for_nn)
    ce = ce_loss(pre, y_for_nn)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%10 == 0:
        print(f'Epoch: {step}/100.............')
        _, predicted = torch.max(pre.data, 1)
        print(f'F1: {f1_score(y_for_nn, predicted)}')
        print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
        print(f'Precision: {precision_score(y_for_nn, predicted)}')
        print(f'Recall: {recall_score(y_for_nn, predicted)}')

print('Final--------------------------')    
_, predicted = torch.max(pre.data, 1)
print(f'F1: {f1_score(y_for_nn, predicted)}')
print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
print(f'Precision: {precision_score(y_for_nn, predicted)}')
print(f'Recall: {recall_score(y_for_nn, predicted)}')

Epoch: 0/100.............
F1: 0.8196092055961323
Accuracy: 0.7011711712651236
Precision: 0.7293915168505806
Recall: 0.9352948687946427
Epoch: 10/100.............
F1: 0.9036414262357573
Accuracy: 0.8502000489106942
Precision: 0.8475196263525633
Recall: 0.9677229558952656


KeyboardInterrupt: 

### Model Testing Bayes

In [ ]:
for i in range(iterations):
    pre_test = model(x_for_nn_test)
    
_, predicted_test = torch.max(pre_test.data, 1)

In [ ]:
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

### Trying out training with batches

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)

In [61]:
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)

In [63]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)

In [90]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [64]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([6136899, 79])

In [65]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([6136899])

In [66]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([1534225, 79])

In [67]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([1534225])

In [68]:
kl_weight = 0.1
for step in range(10):
    pre = model(x_for_nn)
    ce = ce_loss(pre, y_for_nn)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if step%10 == 0:
        print(f'Epoch: {step}/100.............')

    
_, predicted = torch.max(pre.data, 1)
print(f'F1: {f1_score(y_for_nn, predicted)}')
print(f'Accuracy: {accuracy_score(y_for_nn, predicted)}')
print(f'Precision: {precision_score(y_for_nn, predicted)}')
print(f'Recall: {recall_score(y_for_nn, predicted)}')

Epoch: 0/100.............
F1: 0.8410199569940885
Accuracy: 0.7256552535735068
Precision: 0.7256552535735068
Recall: 1.0


In [69]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.8416148118674033
Accuracy: 0.7265414134171976
Precision: 0.7265414134171976
Recall: 1.0


In [144]:
batch_size = 1048576
train_data = TensorDataset(x_for_nn, y_for_nn)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=False)

In [146]:
kl_weight = 0.1
epochs = 10
for epoch in range(epochs):
    start_time = time.process_time()
    avg_loss = 0.
    counter = 0
    for x, y in train_loader:
        counter += 1
        pre = model(x)
        ce = ce_loss(pre, y)
        kl = kl_loss(model)
        cost = ce + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        avg_loss += cost.item()
        if epoch%1 == 0:
            print(f"Epoch {epoch}, Batch {counter}...... Average Loss for Epoch: {avg_loss/counter}")
            _, predicted = torch.max(pre.data, 1)
            print(f'F1: {f1_score(y, predicted)}')
            print(f'Accuracy: {accuracy_score(y, predicted)}')
            print(f'Precision: {precision_score(y, predicted)}')
            print(f'Recall: {recall_score(y, predicted)}')            
    current_time = time.process_time()

Epoch 0, Batch 1...... Average Loss for Epoch: 0.45301762223243713
F1: 0.9067402890959289
Accuracy: 0.8293914794921875
Precision: 0.8293914794921875
Recall: 1.0
Epoch 0, Batch 2...... Average Loss for Epoch: 0.46526767313480377
F1: 0.9059732037637637
Accuracy: 0.8281087875366211
Precision: 0.8281087875366211
Recall: 1.0
Epoch 0, Batch 3...... Average Loss for Epoch: 0.4530884524186452
F1: 0.906014865017131
Accuracy: 0.8281784057617188
Precision: 0.8281784057617188
Recall: 1.0
Epoch 0, Batch 4...... Average Loss for Epoch: 0.4430006369948387
F1: 0.9059572232629743
Accuracy: 0.8280820846557617
Precision: 0.8280820846557617
Recall: 1.0
Epoch 0, Batch 5...... Average Loss for Epoch: 0.4351014494895935
F1: 0.9061335536555546
Accuracy: 0.8283767700195312
Precision: 0.8283767700195312
Recall: 1.0
Epoch 0, Batch 6...... Average Loss for Epoch: 0.4339878062407176
F1: 0.9063959467098306
Accuracy: 0.8288154602050781
Precision: 0.8288154602050781
Recall: 1.0
Epoch 0, Batch 7...... Average Loss for

F1: 0.9365044048851455
Accuracy: 0.8949451446533203
Precision: 0.9379782307238451
Recall: 0.935035203364606
Epoch 3, Batch 6...... Average Loss for Epoch: 0.21851693342129389
F1: 0.9402030588066347
Accuracy: 0.9007234573364258
Precision: 0.9378644248047233
Recall: 0.9425533850762496
Epoch 3, Batch 7...... Average Loss for Epoch: 0.21839136736733572
F1: 0.953747718052176
Accuracy: 0.9204578399658203
Precision: 0.920957992768896
Recall: 0.9889585297973641
Epoch 3, Batch 8...... Average Loss for Epoch: 0.22233273833990097
F1: 0.9553650778930727
Accuracy: 0.9228858947753906
Precision: 0.9180976085367419
Recall: 0.9957860821035068
Epoch 3, Batch 9...... Average Loss for Epoch: 0.220656865172916
F1: 0.9380622887564009
Accuracy: 0.8982000350952148
Precision: 0.9453948345611669
Recall: 0.9308426109747167
Epoch 3, Batch 10...... Average Loss for Epoch: 0.22127765268087388
F1: 0.932748521980683
Accuracy: 0.8947172164916992
Precision: 0.9905266004313619
Recall: 0.8813394055630895
Epoch 3, Batch 1

Epoch 6, Batch 7...... Average Loss for Epoch: 0.17486406862735748
F1: 0.9589414194097235
Accuracy: 0.9296693801879883
Precision: 0.9276751272972481
Recall: 0.9923888168038332
Epoch 6, Batch 8...... Average Loss for Epoch: 0.17381695471704006
F1: 0.9659341477230439
Accuracy: 0.9428186416625977
Precision: 0.9539555782990834
Recall: 0.9782173660377793
Epoch 6, Batch 9...... Average Loss for Epoch: 0.17295365863376194
F1: 0.9680173455242271
Accuracy: 0.9462480545043945
Precision: 0.9545983597750958
Recall: 0.9818189770973211
Epoch 6, Batch 10...... Average Loss for Epoch: 0.172318759560585
F1: 0.9489254572279404
Accuracy: 0.9157800674438477
Precision: 0.9537015185985601
Recall: 0.9441969937619409
Epoch 6, Batch 11...... Average Loss for Epoch: 0.1720787544142116
F1: 0.9587299462630962
Accuracy: 0.9294309616088867
Precision: 0.9301393641497142
Recall: 0.9891338997699498
Epoch 6, Batch 12...... Average Loss for Epoch: 0.17163229609529176
F1: 0.9586533134645779
Accuracy: 0.9293222427368164
P

Precision: 0.9553835642521925
Recall: 0.981814624039596
Epoch 9, Batch 9...... Average Loss for Epoch: 0.16352462602986229
F1: 0.9624559446191118
Accuracy: 0.9356222152709961
Precision: 0.9313934550637087
Recall: 0.9956618198723172
Epoch 9, Batch 10...... Average Loss for Epoch: 0.1623132437467575
F1: 0.9719616868345355
Accuracy: 0.9537811279296875
Precision: 0.9772218047124572
Recall: 0.9667578933315533
Epoch 9, Batch 11...... Average Loss for Epoch: 0.16347378221425143
F1: 0.9444208135814514
Accuracy: 0.9121484756469727
Precision: 0.9922864608624102
Recall: 0.9009605223473398
Epoch 9, Batch 12...... Average Loss for Epoch: 0.16436213006575903
F1: 0.9459572242031399
Accuracy: 0.9142856597900391
Precision: 0.9901215174195107
Recall: 0.9055645863169383
Epoch 9, Batch 13...... Average Loss for Epoch: 0.1630904651605166
F1: 0.969807661775046
Accuracy: 0.9492206573486328
Precision: 0.9557197427741854
Recall: 0.9843171244339777
Epoch 9, Batch 14...... Average Loss for Epoch: 0.1619535269481

In [147]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.9198960533303966
Accuracy: 0.8528816401362974
Precision: 0.8901990848491308
Recall: 0.9516427783902977


In [80]:
df_test = dfs_parquet['Wednesday-28-02-2018']
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Date'] = pd.to_datetime(df_test['Timestamp']).dt.date
df_test['TS_relative'] = (df_test['Timestamp'].astype(int) - 
                             pd.to_datetime(df_test['Date']).astype(int))/ 10**9
df_test = df_test.drop(['Timestamp'], axis = 1)
df_test = df_test.drop(['Date'], axis = 1)
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.fillna(0, inplace=True)
df_test[cat_cols] = df_test[cat_cols].astype('category')
X_test = df_test.drop(['is_allowed'], axis = 1)
y_test = df_test['is_allowed'].astype(int)*1

In [81]:
stdsc = MinMaxScaler()
stdsc.fit(X_test)
X_test_scaled = stdsc.transform(X_test)
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([613071])

In [82]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.7754044128302481
Accuracy: 0.6460768817967251
Precision: 0.8878006693656196
Recall: 0.6882690187431092
